# 临时会话记忆

- LangChain 内置历史记录附加功能
  - `RunnableWithMessageHistory`：在原有链的基础上创建带有历史记录功能的新链
  - `InMemoryChatMessageHistory`：为历史记录提供内存存储

In [16]:
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

In [17]:
model = ChatTongyi(model="qwen3-max")

In [18]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你需要根据会话历史回应用户问题。对话历史："),
        MessagesPlaceholder("chat_history"),
        ("human", "请回答如下问题：{input}")
    ]
)

In [19]:
str_parser = StrOutputParser()

In [20]:
def print_prompt(full_prompt):
    print("="*20)
    print(full_prompt.to_string())
    print("="*20)
    return full_prompt

In [21]:
store = {}
# key就是session，value就是InMemoryChatMessageHistory类对象
# 实现通过会话id获取InMemoryChatMessageHistory类对象
def get_history(session_id):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()

    return store[session_id]

## RunnableWithMessageHistory 方法

- 参数：
  - `base_chain`：被增强的chain
  - `get_history`：获取历史消息的函数
  - `input_messages_key`：用户输入在模板中的占位符
  - `history_messages_key`：历史消息在模板中的占位符

In [22]:
base_chain = prompt | print_prompt | model | str_parser
# 创建一个新的链，对原有链增强功能：自动附加历史消息
conversation_chain = RunnableWithMessageHistory(
    base_chain,     # 被增强的原有chain
    get_history,    # 通过会话id获取InMemoryChatMessageHistory类对象
    input_messages_key="input",             # 表示用户输入在模板中的占位符
    history_messages_key="chat_history"     # 表示用户输入在模板中的占位符
)

In [23]:
session_config = {
    "configurable": {
        "session_id": "user_001"
    }
}

In [24]:
res = conversation_chain.invoke({"input": "小明有2个猫"}, session_config)
print("第1次执行：", res)

res = conversation_chain.invoke({"input": "小刚有1只狗"}, session_config)
print("第2次执行：", res)

res = conversation_chain.invoke({"input": "总共有几个宠物"}, session_config)
print("第3次执行：", res)

System: 你需要根据会话历史回应用户问题。对话历史：
Human: 请回答如下问题：小明有2个猫
第1次执行： 小明有2只猫。请问有什么我可以帮您解答或继续探讨的吗？
System: 你需要根据会话历史回应用户问题。对话历史：
Human: 小明有2个猫
AI: 小明有2只猫。请问有什么我可以帮您解答或继续探讨的吗？
Human: 请回答如下问题：小刚有1只狗
第2次执行： 小刚有1只狗。
System: 你需要根据会话历史回应用户问题。对话历史：
Human: 小明有2个猫
AI: 小明有2只猫。请问有什么我可以帮您解答或继续探讨的吗？
Human: 小刚有1只狗
AI: 小刚有1只狗。
Human: 请回答如下问题：总共有几个宠物
第3次执行： 小明有2只猫，小刚有1只狗，所以他们一共有：

2（猫） + 1（狗） = **3只宠物**。

答：总共有 **3** 个宠物。
